In [1]:
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [24]:
def get_graph_info(graph):
    print("Number of nodes:", graph.number_of_nodes())
    print("Number of edges:", graph.number_of_edges())
    print("Available nodes:", list(graph.nodes))
    print("Available edges:", list(graph.edges))
    print("Connected components:", list(nx.connected_components(graph)))
    print("Node degree:", sorted(d for n, d in graph.degree()))

## 1. Undirected graph

### 1.1 Create graph ausing `add_edge()`

In [28]:
%%time

# create graph by adding edges
undirected_graph = nx.Graph()
undirected_graph.add_edge('A','B')
undirected_graph.add_edge('B','C')
undirected_graph.add_edge('D','A')
undirected_graph.add_edge('E','F')

# display graph info
get_graph_info(undirected_graph)

Number of nodes: 6
Number of edges: 4
Available nodes: ['A', 'B', 'C', 'D', 'E', 'F']
Available edges: [('A', 'B'), ('A', 'D'), ('B', 'C'), ('E', 'F')]
Connected components: [{'B', 'D', 'C', 'A'}, {'F', 'E'}]
Node degree: [1, 1, 1, 1, 2, 2]
Wall time: 998 µs


### 1.2 Create graph using `nx.Graph()` class constructor

In [30]:
%%time

# create graph from edge list
edgelist = [('A', 'B'), ('A', 'D'), ('B', 'C'), ('E', 'F')]
undirected_graph = nx.Graph(edgelist)

# display graph info
get_graph_info(undirected_graph)

Number of nodes: 6
Number of edges: 4
Available nodes: ['A', 'B', 'D', 'C', 'E', 'F']
Available edges: [('A', 'B'), ('A', 'D'), ('B', 'C'), ('E', 'F')]
Connected components: [{'B', 'D', 'C', 'A'}, {'F', 'E'}]
Node degree: [1, 1, 1, 1, 2, 2]
Wall time: 1 ms


### 1.3 Create graph using `nx.from_pandas_edgelist()`

In [42]:
%%time
# create graph from pandas dataframe
relationships = pd.DataFrame({'from': ['A', 'A', 'B', 'E'], 
                              'to':   ['B', 'D', 'C', 'F']})
G = nx.from_pandas_edgelist(relationships, 'from', 'to', create_using=nx.Graph())

# display graph info
get_graph_info(undirected_graph)

Number of nodes: 6
Number of edges: 4
Available nodes: ['A', 'B', 'D', 'C', 'E', 'F']
Available edges: [('A', 'B'), ('A', 'D'), ('B', 'C'), ('E', 'F')]
Connected components: [{'B', 'D', 'C', 'A'}, {'F', 'E'}]
Node degree: [1, 1, 1, 1, 2, 2]
Wall time: 1.96 ms


In [79]:
def draw_graph(graph, figsize=(10,8), title=""):
    np.seed(2021)
    fig, ax = plt.subplots(figsize=figsize)
    nx.draw(graph, with_labels=True, font_color='white')
    plt.title(title)
    plt.show()
draw_graph(undirected_graph, title="Undirected graph")

AttributeError: module 'numpy' has no attribute 'seed'